In [ ]:
import openpyxl
import pandas as pd
import os
import csv
from tkinter import Tk, filedialog

#変数にある情報をエクセルに出力する
def ALARMCODE(excel_name,machine_name,language,inisial_code):
    wb_1=openpyxl.load_workbook(excel_name)
    ws_1=wb_1[machine_name]
    file_name = "book1.xlsx"
    wb_2 = openpyxl.load_workbook(file_name)
    ws_2=wb_2.worksheets[0]  
    max_row=ws_1.max_row

    # 言語によって列番号を設定
    if language =="INFO (POLISH)":
            excel_column=3
    elif language =="INFO (english)":
            excel_column=4
    else:
        excel_column=5

    #アラームコードの生成と書き込み
    for i in range(1,1025):
        alarm_name = str(ws_1.cell(row=i+1,column=excel_column).value)
        CH_code = f"{inisial_code + (i-1) // 16}"
        BIT_code = f"{(i-1) % 16:02}"
        alarm_code = f"{i},H{CH_code}.{BIT_code}_{alarm_name}"
        
        #アラームコードフォーマット調整
        if len(alarm_code) < 38:
            alarm_code = alarm_code.ljust(63, ' ')  # 長さが63になるように空白で埋める
            alarm_code = alarm_code[:62] + "/"      # 63文字目に "/" を挿入する
        ws_2.cell(row=i,column=1).value = alarm_code
    
    #名前を付けて保存
    #name_wb_1 = f"{machine_name}{language}.xlsx"

     # "Temp"フォルダにファイルを保存
    temp_folder = "Temp"
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)

    name_wb_1 = os.path.join(temp_folder, f"{machine_name}{language}.xlsx")
    wb_2.save(name_wb_1)

    # ワークブックを閉じる
    wb_1.close()
    wb_2.close()
    
    return  name_wb_1 , machine_name

#関数alarmcodeで出力したエクセルファイル（excel_file）を
#テキストファイル(n__alm.txt※nは設備番号)に変換する
def TXT(excel_file, machine_name, ls_machine_name,language):
    n = "00"  # 文字列として初期化
    for i in range(len(ls_machine_name)):
        if machine_name == ls_machine_name[i]:
            n = f"{i+1:02}"  # 2桁のゼロ埋め
    if n == "00":  # nが初期値のままなら
        n = "fail"
    # エクセルファイルを読み込む
    workbook = openpyxl.load_workbook(excel_file)
    sheet = workbook.active
    
    # 保存先ディレクトリ "folder" の作成（存在しない場合は作成）
    if language == "INFO (POLISH)":
        folder_path = 'PO'  # 任意のフォルダ名
    elif language == "INFO (english)":
        folder_path = 'EN'
    else :
        folder_path = 'JP'
        
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # テキストファイルに書き出す（保存先を"folder"に変更）
    file_path = os.path.join(folder_path, f"{n}_alm.txt")
    with open(file_path, 'w', encoding='utf-16le') as file:
        for row in sheet.iter_rows(min_col=1, max_col=1, values_only=True):
            cell_value = row[0]  # 1列目の値
            if cell_value is not None:  # Noneチェックを追加
                #file.write(str(cell_value) + '\n') 
                file.write(f"{cell_value}\n")
    print(f"Text file has been created. file path：{file_path}")
    
    # フォルダ整理の為Excelファイルを削除
    workbook.close()  # ワークブックを閉じる

#####main処理#####
# 新しいワークブック"book1.xlsx"を作成
wb = openpyxl.Workbook()
sheet = wb.active
sheet.title = "Sheet1"
# ワークブックを保存
wb.save("book1.xlsx")

# tkinterのウィンドウを非表示に設定
root = Tk()
root.withdraw()  # メインウィンドウを表示しないように設定

# 読込みExcelファイルの選択
print("Select the Excel file for the alarm information.")
excel_name = filedialog.askopenfilename(
    title="Select the Excel file",
    filetypes=[("Excel files", "*.xlsx *.xls")]
)

# 読込みExcelファイルが選択されなかった場合
if not excel_name:
    print("Processing was aborted.")
    root.destroy()
    exit()  # プログラムを終了

# 読込みCSVファイルの選択
print("Select the CSV file for the machine list.")
machine_list_name = filedialog.askopenfilename(
    title="Select the CSV file",
    filetypes=[("CSV files", "*.csv")]
)

# 読込みCSVファイルが選択されなかった場合
if not machine_list_name:
    print("Processing was aborted.")
    root.destroy()
    exit()  # プログラムを終了

# tkinterを閉じる
root.destroy()

# 選択されたファイル名を確認の為表示
print(f"Selected Excel file: {excel_name}")
print(f"Selected CSV file: {machine_list_name}")

#リスト初期化
ls_ws = []
ls_machine_name = []
ls_language = []
ls_5s_excel = []

#ワークブックを指定しシート名をリストへ格納#
workbook = openpyxl.load_workbook(excel_name)
ls_ws = workbook.sheetnames

ls_language = ["INFO (POLISH)","INFO (english)","INFO (JAPANESE)"]

# 設備リストから設備名を取得
with open(machine_list_name, newline='') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    if len(header) > 1:
        ls_machine_name.append(header[1])  # 先頭設備のみ列名からリストへ格納
    for row in reader:
        if len(row) > 1:
            ls_machine_name.append(row[1])

#ワークブックの各シートよりシート名、先頭HR番号を取得し関数へ渡す。
for i in ls_language:
    for j in range(len(ls_ws)):
        sheet = workbook[ls_ws[j]]
        cell_value = sheet['B2'].value
        inisial_code = cell_value // 100
        #inisial_code=26
        excel_file,machine_name = ALARMCODE(excel_name,ls_ws[j],i,inisial_code)
        print(f"{excel_file}, {machine_name}, {inisial_code}")
        TXT(excel_file,machine_name,ls_machine_name,i)
        ls_5s_excel.append(excel_file)    #フォルダ整理の為エクセル名をリスト化

#フォルダ整理の為Excelファイルを削除
for i in ls_5s_excel:
    if os.path.exists(i):  # ファイルが存在するか確認
        os.remove(i)
        print(f"Excel file has been deleted. File path：{excel_file}")
print("Processing completed.")